In [1]:
import io, os, sys, types
from Utils import NotebookFinder
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from Configuration import printPeriodic,setDbgPrint,null
from data_Generator import dataGenerator


sys.meta_path.append(NotebookFinder())

importing Jupyter notebook from Configuration.ipynb
importing Jupyter notebook from data_Generator.ipynb
importing Jupyter notebook from runs.ipynb


In [2]:
class runTest():
    def __init__(self,**kwargs):
        self.actions = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
        self.n_actions = len(self.actions)
        self.n_features = 76
        self.built_net()
        self.TPcount = 0
        self.FPcount = 0
        self.TNcount =0
        self.FNcount = 0
        # sess = tf.InteractiveSession()
        # self.saver.restore(sess, 'my_net/my_test_model.ckpt-20000')  # Load parameter
        self.params = []
        self.cost_his = []
        self.time = 0
        self.flag = 0
        self.Kcount = kwargs.get('Kcount')
        self.list = dataGenerator(self.flag,self.Kcount)
        # self.label = data_Generator_label(self.flag)
        self.reward_history = []
        self.count_history =[]
        self.Pr_history = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
        self.Rc_history = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
        self.F1_history = [[], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
        self.count=np.zeros((15, 4))
        self.time_env_state = self.time_env_state ={"next":{},"current":{}}
        target_net = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_net')
        saver1 = tf.train.Saver(target_net)
        self.sess = tf.Session()
        #Upload model parameters
       # model_file=tf.train.latest_checkpoint('my_net/')
        #saver1.restore(self.sess,model_file)
        saver1.restore(self.sess, "./my_net/target_net.ckpt-370000")
    def built_net(self):
        tf.reset_default_graph()
        # Reconstruction neural network model.Only one of the neural networks is kept here.
        # c_names, n_l1, w_initializer, b_initializer = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES], 10, tf.random_normal_initializer(0., 0.3), tf.constant_initializer(0.1)
        n_l1 = 10
        self.env_state_ = tf.placeholder(tf.float32, [None, self.n_features], name='env_state_')
        with tf.variable_scope('target_net'):
            c_names = ['target_net_params', tf.GraphKeys.GLOBAL_VARIABLES]

            with tf.variable_scope(
                    'l1'):
                w1 = tf.get_variable('w1', [self.n_features, n_l1], collections=c_names)
                b1 = tf.get_variable('b1', [1, n_l1], collections=c_names)
                l1 = tf.nn.tanh(tf.matmul(self.env_state_, w1) + b1)
            with tf.variable_scope('l2'):
                w2 = tf.get_variable('w2', [n_l1, self.n_actions], collections=c_names)
                b2 = tf.get_variable('b2', [1, self.n_actions], collections=c_names)
                self.q_next = tf.matmul(l1, w2) + b2


    def choose_action(self, env_state):
        env_state = env_state[np.newaxis, :]
        actions_value = self.sess.run(self.q_next,feed_dict={self.env_state_: env_state})
        action = np.argmax(actions_value)
        return action

    def data(self):
        Network_data = self.list[self.time]
        return Network_data

    def update_State(self):
        self.time_env_state["next"]["feature"] = self.data()[0:76]
        self.time_env_state["next"]["label"] = self.data()[76]
        self.time += 1
        return self.time_env_state["next"]


    def reset(self):
        self.time_env_state["next"] = self.update_State()
        return self.time_env_state["next"]
    def reward(self, action,state):

        self.time_env_state["current"] = state
        #print(self.time_env_state["current"] )
        label = self.time_env_state["current"]["label"]
        if action == self.time_env_state["current"]["label"]:
            reward = 10
        else:
            reward = 0


        return reward,label

    def step(self, action,state):

        value = 0
        min_value = np.Inf
        action_key = ""

        #self.log('{}: action {} has min. value {}\n'.format(self.time, action_key, min_value), period=self.statusPeriod,
        #         counter=self.time)
        # next_state = action
        reward,label = self.reward(action,state)
        self.reward_ = reward

        self.count_history.append(self.reward_)
        # self.correct_rate.append(self.count/(len(self.count_history)))

        self.state = self.update_State()

        return  self.time_env_state["next"], reward, label

    def run_(self,cfg=None,):
        numEpisodes = cfg['numEpisodesTest']
        maxSteps = cfg['maxStepsTest']
        dbgPrint = cfg.get('dbgPrint', null)
        if self.Kcount == 0:
            maxSteps = 943583
        elif self.Kcount == 1:
            maxSteps = 943579
        else:
            maxSteps = 943574
        statusPeriod = cfg.get('statusPeriod', 1)
        for episode in range(1,numEpisodes+1):
            # initial observation
            step = 0
            env_state_1 = self.reset()
            while step < maxSteps:
                #print('{}: current env = {}\n'.format(self.time, self.time_env_state))
                env_state = env_state_1["feature"]
                env_state = np.hstack(env_state)
                action = self.choose_action(env_state)
                # print('{}: action_ = {}, observation = {}\n'.format(self.time, action_, observation))
                observation_,reward,label = self.step(action,env_state_1)

                label = int(label)
                action = int(action)
                
                counti =0
                for i in [label,action]:
                   # if i==1:
                        # print("label= {},action = {}".format(label,action))
                        #print("i = {}".format(i))
                    
                    if label == i and action == i:  # TP
                        self.count[i][0]+= 1
                        #print("{}kkkkkkkkkkkkkkkkkkkkkkkk".format(i))
                        if counti == 1:
                            self.count[i][0] -=1
                        counti += 1

                        # print('kkkkkk',self.count[i][0])
                    elif action == i and label != i:  # FP
                        self.count[i][1] += 1
                        # print('kkkkkk',self.count[i][1])

                    elif action != i and label != i:  # TN
                        self.count[i][2] += 1
                        #print('kkkkkk',self.count[i][2])
                    elif action != i and label == i:  # FN
                        self.count[i][3] += 1
                       # print('kkkkkk',self.count[i][3])
                   #print(self.count[i][0] + self.count[i][3],self.count[i][0] + self.count[i][1])
                    
                    if ((self.count[i][0] + self.count[i][3]) != 0 and (self.count[i][0] + self.count[i][1]) != 0):
                        Pr = self.count[i][0] / (self.count[i][0] + self.count[i][1])
                        Rc = self.count[i][0] / (self.count[i][0] + self.count[i][3])
                        F1 = 2 / (1 / Pr + 1 / Rc)
                        self.Rc_history[i].append(self.count[i][0]/(self.count[i][0]+self.count[i][3]))
                        self.Pr_history[i].append(self.count[i][0]/(self.count[i][0]+self.count[i][1]))
                        #print('{}___Rc = {},Pr = {},F1 = {}'.format(i,Rc,Pr,F1))
                        self.F1_history[i].append(F1)
                        


                env_state_1 =  observation_
                step += 1
        plt.figure()
        for i in range(1,16):
            plt.subplot(1,3,1)
            plt.plot(self.Pr_history[i-1], '.-')
            plt.xlabel('Test Step')
            plt.ylabel('Pr_{}'.format(i-1))

            plt.subplot(1, 3, 2)
            plt.plot(self.Rc_history[i-1], '.-')
            plt.xlabel('Test Step')
            plt.ylabel('Rc_{}'.format(i-1))

            plt.subplot(1, 3, 3)
            plt.plot(
                self.F1_history[i-1], '.-')
            plt.xlabel('Test Step')
            plt.ylabel('F1')
            plt.grid(True)
            plt.show()
        plt.figure()
        plt.plot(
            np.divide( np.cumsum([1 if i >= 10 else 0 for i in self.count_history]), np.arange(len(self.count_history)) + 1),
            '.-')
        plt.xlabel('Training Step')
        plt.ylabel('Correct rate')
        plt.grid(True)
        plt.show()